In [1]:
import glob
import torch

pth_list = glob.glob("./models/mosquitoes/*.pth")
pth_list.sort()

for pth in pth_list:
    print("Current File : ", pth)
    acc = torch.load(pth)['acc']
    print(f" Accuracy : {acc*100:.2f}%")
    
del pth_list

Current File :  ./models/mosquitoes/best_resnet18_x16_True.pth
 Accuracy : 92.97%
Current File :  ./models/mosquitoes/best_resnet18_x1_True.pth
 Accuracy : 98.21%
Current File :  ./models/mosquitoes/best_resnet18_x2_True.pth
 Accuracy : 98.04%
Current File :  ./models/mosquitoes/best_resnet18_x4_True.pth
 Accuracy : 96.49%
Current File :  ./models/mosquitoes/best_resnet18_x8_True.pth
 Accuracy : 96.01%
Current File :  ./models/mosquitoes/best_resnet34_x16_True.pth
 Accuracy : 92.77%
Current File :  ./models/mosquitoes/best_resnet34_x1_True.pth
 Accuracy : 97.94%
Current File :  ./models/mosquitoes/best_resnet34_x2_True.pth
 Accuracy : 98.14%
Current File :  ./models/mosquitoes/best_resnet34_x4_True.pth
 Accuracy : 96.96%
Current File :  ./models/mosquitoes/best_resnet34_x8_True.pth
 Accuracy : 95.71%
Current File :  ./models/mosquitoes/best_resnet50_x16_True.pth
 Accuracy : 94.70%
Current File :  ./models/mosquitoes/best_resnet50_x1_True.pth
 Accuracy : 96.99%
Current File :  ./models/

In [2]:
import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim, utils
from torchvision import models, datasets, transforms

import PIL.Image as Image

from tqdm import tqdm
import random
import cv2
import os
import lrResnet

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10,10)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device ...", device)

Using device ... cuda


In [3]:
def get_model(model, pretrained=False, down_scale=4, num_classes = 18):
    if model == 'resnet50':
        if down_scale == 1:
            net = models.resnet50(pretrained=pretrained, num_classes=num_classes)
        else:
            net = lrResnet.resnet50_LR(scale=down_scale, pretrained=pretrained)
        net.fc = nn.Linear(in_features=2048, out_features=num_classes)
    elif model == 'resnet34':
        if down_scale == 1:
            net = models.resnet34(pretrained=pretrained, num_classes=num_classes)
        else:
            net = lrResnet.resnet34_LR(scale=down_scale, pretrained=pretrained)
        net.fc = nn.Linear(in_features=512, out_features=num_classes)
        
    elif model == 'resnet18':
        if down_scale == 1:
            net = models.resnet18(pretrained=pretrained, num_classes=num_classes)
        else:
            net = lrResnet.resnet18_LR(scale=down_scale, pretrained=pretrained)
        net.fc = nn.Linear(in_features=512, out_features=num_classes)
    return net
model_name = 'resnet50'
down_scale = 16
net_t = get_model(model_name, pretrained=False, down_scale=down_scale, num_classes=18)

w_dict = torch.load(f"./models/birds/best_{model_name}_x{down_scale}_True.pth")['net']
net_t.load_state_dict(w_dict)
net_t = net_t.to(device)

In [4]:
for i in [1,2,4,8,16]:
    
    init_scale = 1.15

    test_scale = i

# -------->> For mosquitoes >>--------
#     transforms_test = transforms.Compose([
#         transforms.Resize(224//down_scale),
#         transforms.ToTensor()
#     ])
#     test_dataset = datasets.ImageFolder("/media/data/MosquitoDL/Test/", transform=transforms_test)
# ------------------------------------

# -------->> For ILSVRC Birds >>----------
    transforms_test = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.Resize(224//test_scale),
        transforms.ToTensor(),
    ])
    test_dataset = datasets.ImageFolder("../dataset/ILSVRC_Birds/val", transform=transforms_test)
# ----------------------------------------    


    test_loader = utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True)

    def evaluate(net ,test_loader):
        net.eval()

        n_count = 0
        n_corrects = 0

        for j, data in enumerate(test_loader):


            batch, label = data[0].to(device), data[1].to(device)

            pred = net(batch)

            n_corrects += torch.sum(torch.argmax(pred, dim=1) == label).item()
            n_count += label.shape[0]

        test_accuracy = n_corrects/n_count

        return test_accuracy, net
    with torch.no_grad():
        acc, net_t = evaluate(net_t, test_loader)

        print(f"[{i}] Accuracy: {acc*100:.2f}%")

[1] Accuracy: 17.67%
[2] Accuracy: 38.00%
[4] Accuracy: 63.56%
[8] Accuracy: 78.00%
[16] Accuracy: 72.56%
